In [3]:
import pandas as pd
from pathlib import Path

csvpath= "Resources/Airbnb_cleaned.csv"

df= pd.read_csv(csvpath)

df = df.drop(columns=['Unnamed: 0'])

In [4]:
from pymongo import MongoClient

# Data cleaning and transformation

df.columns = [col.strip().replace(" ", "_").lower() for col in df.columns]  

df.fillna(0, inplace=True)

df['name'] = df['name'].str.title()

# Connect to MongoDB and insert the data
client = MongoClient("mongodb://localhost:27017/")  
db = client["airbnb_db"] 
collection = db["listings"]  

# Convert the DataFrame to a dictionary and insert into MongoDB
data_dict = df.to_dict(orient='records')  
collection.insert_many(data_dict)  

print(f"{len(data_dict)} records inserted into MongoDB.")  

100530 records inserted into MongoDB.


# Airbnb Map Section

In [5]:
# Listing count by NYC neighborhood dataframe
listing_count = df.groupby('neighbourhood')['id'].count().reset_index(name='number of listings')
listing_count.rename(columns={'neighbourhood': 'neighborhood'}, inplace=True)
listing_count

,neighborhood,number of listings
0,Allerton,94
1,Arden Heights,9
2,Arrochar,50
3,Arverne,221
4,Astoria,1831
...,...,...
219,Windsor Terrace,322
220,Woodhaven,189
221,Woodlawn,29
222,Woodrow,3


In [6]:
# NYC neighborhood stats CSV (/Resources)
avg_df = df.groupby(['neighbourhood group', 'neighbourhood'])[['price', 'service fee', 'review rate number']].mean().reset_index()
avg_df.rename(columns={'neighbourhood group': 'borough', 'neighbourhood': 'neighborhood'}, inplace=True) 
avg_df['price'] = avg_df['price'].astype('int64')
avg_df['service fee'] = avg_df['service fee'].astype('int64')
avg_df['review rate number'] = avg_df['review rate number'].round(2)
merged_avg_df = pd.merge(avg_df, listing_count, on='neighborhood', how='inner')
merged_avg_df.to_csv('Resources/neighborhood_stats.csv', index=False)
merged_avg_df

KeyError: 'neighbourhood group'